## SageMaker endpoint
To deploy the model you previously trained, you need to create a Sagemaker Endpoint. This is a hosted prediction service that you can use to perform inference.

### Finding the model
This notebook uses a stored model if it exists. If you recently ran a training example that use the %store% magic, it will be restored in the next cell.

Otherwise, you can pass the URI to the model file (a .tar.gz file) in the model_data variable.

You can find your model files through the SageMaker console by choosing Training > Training jobs in the left navigation pane. Find your recent training job, choose it, and then look for the s3:// link in the Output pane. Uncomment the model_data line in the next cell that manually sets the model's URI.

In [ ]:
# Retrieve a saved model from a previous notebook run's stored variable
%store -r model_data

# If no model was found, set it manually here.
# model_data = 's3://sagemaker-us-west-2-XXX/pytorch-smdataparallel-mnist-2020-10-16-17-15-16-419/output/model.tar.gz'

print("Using this model: {}".format(model_data))

### Create a model object
You define the model object by using SageMaker SDK's PyTorchModel and pass in the model from the estimator and the entry_point. The endpoint's entry point for inference is defined by model_fn as seen in the following code block that prints out inference.py. The function loads the model and sets it to use a GPU, if available.

In [ ]:
!pygmentize inference.py

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()

from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(model_data=model_data,
                        entry_point='inference.py', role=role, framework_version='1.6.0', py_version='py3')

#### Deploy the model on an endpoint
You create a predictor by using the model.deploy function. You can optionally change both the instance count and instance type.

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Test the model
You can test the depolyed model using samples from the test set.

In [ ]:

# Download the test set
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from train import TileDataset

import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='sagemaker-us-east-2-318322629142'

tiles_key = 'train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/train_tiles/'
tiles_dir = 's3://{}/{}'.format(bucket, tiles_key)

# dataset_csv_key = ‘panda_dataset.csv’
# dataset_csv_dir = 's3://{}/{}'.format(bucket, dataset_csv_key)

# df = pd.read_csv(dataset_csv_dir)
# df['isup_grade'] = df['isup_grade'].replace([1,2], 0)
# df['isup_grade'] = df['isup_grade'].replace([3,4,5], 1)

# tiles_dict = {}
# for image in os.listdir(tiles_dir):
#     tiles_dict[image.split('_')[0]] = tiles_dict.get(image.split('_')[0], 0) + 1

# tiles_df = list(tiles_dict.keys())

# new_tiles_df = []
# for i in range(len(tiles_df)):
#   row = df.loc[df['image_id'] == tiles_df[i]]
#   new_tiles_df.append(row.to_dict())

# tiles_df = pd.DataFrame(new_tiles_df)

# # Use only half of the data
# tiles_df = np.array_split(df, 2)

# # Train-test split
# train_df, test_df = train_test_split(tiles_df[0], test_size=0.2)

test_df = pd.read_csv('s3://{}/{}'.format(bucket, 'test_df'))

transform_train = transforms.Compose([transforms.RandomHorizontalFlip(0.5),
                                      transforms.RandomVerticalFlip(0.5),
                                      transforms.ToTensor()])

test_set = TileDataset(tiles_dir, test_df, 12, transform=transform_train)

batch_size = 1
test_loader = data_utils.DataLoader(test_set, batch_size, shuffle=False, num_workers=0)

# data, bag_label = iter(test_loader).next()
# if cuda:
#   data, bag_label = data.cuda(), bag_label.cuda()
# data, bag_label = Variable(data), Variable(bag_label)

In [ ]:
# Send the sampled images to endpoint for inference
# error, predicted = predictor.calculate_classification_error(data, bag_label)
predictions = []
true_labels = []
for batch_idx, (data, label) in enumerate(test_loader):
  _, Y_hat, _ = predictor.predict(data)
  predictions.append(int(Y_hat))
  true_labels.append(label)


# print("Predictions: ")
# print(predicted.tolist())

#### Cleanup
If you don't intend on trying out inference or to do anything else with the endpoint, you should delete it.

In [ ]:
predictor.delete_endpoint()